In [25]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision.transforms import PILToTensor, ToTensor
import cv2
from keras.utils import to_categorical

In [115]:
video = cv2.VideoCapture('nine_hour_video\Video.mp4')
c = 0
while True:
    c += 1
    ret, frame = video.read()
    cv2.imshow('part', frame[:, frame.shape[1] // 2:])
    if cv2.waitKey(1) & 0xFF == ord('q') or c == 60 * 25:
       break

video.release()
cv2.destroyAllWindows()

In [71]:
dict_type = {'простой': 0, 'вынужденная': 1, 'сварка': 2, 'перерыв': 3}
vid_len = 5
left_pd = pd.read_csv('csv\LabelsLeft_five_hour_video.csv')
right_pd = pd.read_csv('csv\LabelsRight_five_hour_video.csv')
right_pd['sec'] = right_pd['time'].apply(lambda x: (int(x.split(':')[0]) * 3600 + int(x.split(':')[1]) * 60 + int(x.split(':')[2])))
right_pd['new_type'] = right_pd['type'].apply(lambda x: dict_type[x])
left_pd['sec'] = left_pd['time'].apply(lambda x: (int(x.split(':')[0]) * 3600 + int(x.split(':')[1]) * 60 + int(x.split(':')[2])))
left_pd['new_type'] = left_pd['type'].apply(lambda x: dict_type[x])
left = np.zeros(60 * 60 * vid_len)
right = np.zeros(60 * 60 * vid_len)
for second, typ in left_pd[['sec', 'new_type']].to_numpy():
    left[second:] = typ
for second, typ in right_pd[['sec', 'new_type']].to_numpy():
    right[second:] = typ

In [90]:
def get_valid_by_sec(path, sec, right_y, left_y):
    video = cv2.VideoCapture(path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_skip = int(fps)
    c = 0
    data_X = []
    data_y = []
    current_frame = 0
    while True:
        ret, frame = video.read()
        left_img = frame[:, :frame.shape[1] // 2]
        right_img = frame[:, frame.shape[1] // 2:]
        data_X.append(left_img)
        data_X.append(right_img)
        data_y.append(left_y[c])
        data_y.append(right_y[c])
        current_frame += frame_skip
        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        if c == sec - 1:
            break
        c += 1
    video.release()
    return np.array(data_X), np.array(data_y)

In [91]:
get_valid_by_sec('five_hour_video\Video.mp4', 20, right, left)

(array([[[[171, 172, 162],
          [181, 182, 172],
          [182, 183, 173],
          ...,
          [232, 226, 200],
          [247, 242, 220],
          [102,  97,  75]],
 
         [[170, 171, 161],
          [181, 182, 172],
          [182, 183, 173],
          ...,
          [249, 243, 217],
          [255, 255, 238],
          [168, 163, 141]],
 
         [[173, 174, 164],
          [182, 183, 173],
          [181, 182, 172],
          ...,
          [252, 246, 220],
          [242, 237, 215],
          [190, 185, 163]],
 
         ...,
 
         [[  6,  15,  16],
          [  0,   6,   7],
          [  6,  15,  16],
          ...,
          [ 63,  31,  37],
          [ 70,  36,  42],
          [ 70,  36,  42]],
 
         [[ 16,  24,  25],
          [  0,   6,   7],
          [  0,   0,   1],
          ...,
          [ 33,  37,  37],
          [ 40,  42,  42],
          [ 40,  42,  42]],
 
         [[ 13,  21,  22],
          [  0,   7,   8],
          [  0,   6,   7],
   

In [104]:
def one_epoch_train(model, optimizer, loss_function, data):
    X = torch.from_numpy(data[0])
    y = data[1]
    model.train()
    pred = model(X)
    loss = loss_function(pred, y)
    print(f"Current loss = {loss.item()}")
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

In [113]:
model = nn.Sequential(
    #nn.Conv2d(3, 4, 3),
    #nn.ReLU(),
    #nn.MaxPool2d(2, 2),

    nn.Flatten(-1),

    nn.BatchNorm2d(1080), 
    nn.Flatten()
)
optim = torch.optim.SGD(model.parameters(), lr=1e-1)
loss = nn.CrossEntropyLoss()   

In [114]:
one_epoch_train(model, optim, loss, get_valid_by_sec('five_hour_video\Video.mp4', 20, right, left))

RuntimeError: "batch_norm" not implemented for 'Byte'